In [1]:
import os
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from google.colab import drive
drive.mount('/content/drive')

# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Read the feature file into a DataFrame
df = pd.read_csv(feature_files[0])

list_of_column_names = list(df.columns.drop(['Arousal_Value', 'Valence_Value', 'frameTime']))

print(list_of_column_names)
print(len(list_of_column_names))

! pip install opensmile
import pandas as pd
import opensmile

# Initialize OpenSMILE
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

# Process the audio file
y = smile.process_file('/content/drive/MyDrive/FYP_dataset/CNN_Features/abc.wav')

# Set Pandas options to display more rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Print the extracted features
smile_features= set((y.keys()))
print(smile_features)

set_our_features=set(list_of_column_names)
print(set_our_features)

dont_have_features_set=set_our_features-smile_features
print(dont_have_features_set)

dont_have_features_list=list(dont_have_features_set)
print(len(dont_have_features_list))

smile_features_have_in_the_dataset=set_our_features - dont_have_features_set
print(smile_features_have_in_the_dataset)

smile_features_have_in_the_dataset=list(smile_features_have_in_the_dataset)
print(len(smile_features_have_in_the_dataset))

drop_list=[]
#drop_list=dont_have_features_list
for i in range(0,len(dont_have_features_list)):
  drop_list.append(dont_have_features_list[i])

drop_list.append('Arousal_Value')
drop_list.append('Valence_Value')
drop_list.append('frameTime')

print(drop_list)
print(len(drop_list))

print(len(feature_files))

# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(drop_list, axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_v = []
mae_v = []
rmse_v = []
r2_v = []
f1_v = []
accuracy_v = []
precision_v = []
recall_v = []

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_reshaped):
    X_train, X_val = X_reshaped[train_index], X_reshaped[test_index]
    y_v_train, y_v_val = y_v[train_index], y_v[test_index]

    # Split the validation set further into val and test sets
    val_size = len(X_val) // 2
    X_val, X_test = X_val[:val_size], X_val[val_size:]
    y_v_val, y_v_test = y_v_val[:val_size], y_v_val[val_size:]

    # Build the BiLSTM model for Arousal
    model_v = Sequential()
    model_v.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model_v.add(Flatten())

    # Add DNN layers with dropout (modify as needed)
    dnn_layers = [512, 256, 128,64,32]  # Modify this list as needed

    for layer_size in dnn_layers:
        model_v.add(Dense(layer_size, activation='relu'))
        # Add dropout if needed: model_a.add(Dropout(dropout_rate))

    model_v.add(Dense(1))
    model_v.compile(loss='mse', optimizer=Adam(learning_rate=0.001))

    # Train the Arousal model
    model_v.fit(X_train, y_v_train, validation_data=(X_val, y_v_val), epochs=25, batch_size=32)

    # Evaluate the model on the test set
    predictions_v = model_v.predict(X_test)

    # Reshape predictions_a to match the shape of y_a_test
    predictions_v = predictions_v.reshape(y_v_test.shape)

    # Calculate additional evaluation metrics
    mse_v.append(mean_squared_error(y_v_test, predictions_v))
    mae_v.append(mean_absolute_error(y_v_test, predictions_v))
    rmse_v.append(np.sqrt(mean_squared_error(y_v_test, predictions_v)))
    r2_v.append(r2_score(y_v_test, predictions_v))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_v = (predictions_v >= threshold).astype(int)
    y_v_test_binary = (y_v_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_v.append(f1_score(y_v_test_binary, binary_predictions_v))
    accuracy_v.append(accuracy_score(y_v_test_binary, binary_predictions_v))
    precision_v.append(precision_score(y_v_test_binary, binary_predictions_v))
    recall_v.append(recall_score(y_v_test_binary, binary_predictions_v))

# Calculate average performance across all folds
average_mse_v = np.mean(mse_v)
average_mae_v = np.mean(mae_v)
average_rmse_v = np.mean(rmse_v)
average_r2_v = np.mean(r2_v)
average_f1_v = np.mean(f1_v)
average_accuracy_v = np.mean(accuracy_v)
average_precision_v = np.mean(precision_v)
average_recall_v = np.mean(recall_v)

print(f'Average Arousal MSE: {average_mse_v:.4f}')
print(f'Average Arousal MAE: {average_mae_v:.4f}')
print(f'Average Arousal RMSE: {average_rmse_v:.4f}')
print(f'Average Arousal R2 Score: {average_r2_v:.4f}')
print(f'Average Arousal F1 Score: {average_f1_v:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_v:.4f}')
print(f'Average Arousal Precision: {average_precision_v:.4f}')
print(f'Average Arousal Recall: {average_recall_v:.4f}')

# Define the directory path to save the model
model_save_path = '/content/drive/MyDrive/FYP_dataset/models'  # Modify as needed

# Save the Arousal model to the specified directory
model_v.save(os.path.join(model_save_path, 'valence_model.h5'))

Mounted at /content/drive
['F0final_sma_stddev', 'F0final_sma_amean', 'voicingFinalUnclipped_sma_stddev', 'voicingFinalUnclipped_sma_amean', 'jitterLocal_sma_stddev', 'jitterLocal_sma_amean', 'jitterDDP_sma_stddev', 'jitterDDP_sma_amean', 'shimmerLocal_sma_stddev', 'shimmerLocal_sma_amean', 'logHNR_sma_stddev', 'logHNR_sma_amean', 'audspec_lengthL1norm_sma_stddev', 'audspec_lengthL1norm_sma_amean', 'audspecRasta_lengthL1norm_sma_stddev', 'audspecRasta_lengthL1norm_sma_amean', 'pcm_RMSenergy_sma_stddev', 'pcm_RMSenergy_sma_amean', 'pcm_zcr_sma_stddev', 'pcm_zcr_sma_amean', 'audSpec_Rfilt_sma[0]_stddev', 'audSpec_Rfilt_sma[0]_amean', 'audSpec_Rfilt_sma[1]_stddev', 'audSpec_Rfilt_sma[1]_amean', 'audSpec_Rfilt_sma[2]_stddev', 'audSpec_Rfilt_sma[2]_amean', 'audSpec_Rfilt_sma[3]_stddev', 'audSpec_Rfilt_sma[3]_amean', 'audSpec_Rfilt_sma[4]_stddev', 'audSpec_Rfilt_sma[4]_amean', 'audSpec_Rfilt_sma[5]_stddev', 'audSpec_Rfilt_sma[5]_amean', 'audSpec_Rfilt_sma[6]_stddev', 'audSpec_Rfilt_sma[6]_am

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
